In [1]:
import re #pacchetto per espressioni regolari
import os #pacchetto per muoversi nelle cartelle
import sys
sys.path.append('..')  # Add parent directory to the Python path
from utils.helpers import *


In [6]:
train_path = 'data/training.txt'
test_path = 'data/test.txt'

out_dir = 'data/profiling_in'

In [3]:
split = get_split(train_path)
split

'training'

In [4]:

# Necessario eitchettare il blind test

# Read gold.txt file and extract gender values
with open("data/gold.txt", "r") as gold_file:
    gold_data = gold_file.readlines()

# Extract gender labels
gender_list = [line.split()[1].strip() for line in gold_data]

# Read test.txt file
with open("data/test.txt", "r") as test_file:
    test_data = test_file.readlines()

# Replace "?" with gender labels sequentially
updated_test_data = []
gender_index = 0  # Index to track which gender label to use
in_doc = False
for line in test_data:
    if '<doc id="' in line:
        in_doc = True
        if '?' in line:
            label = gender_list[gender_index]
            updated_line = line.replace("?", label)
            gender_index += 1
        else:
            updated_line = line
    elif "</doc>" in line:
        in_doc = False
        updated_line = line
    elif in_doc:
        # If inside a <doc>...</doc> block, continue appending the lines
        updated_line = line.replace("?", label)
    else:
        updated_line = line
    updated_test_data.append(updated_line)

# Write the updated test data to a new file
with open("data/test.txt", "w") as updated_test_file:
    updated_test_file.writelines(updated_test_data)


In [7]:
for file_path in [train_path, test_path]:
    split = get_split(file_path)

    for line in open(file_path, 'r'):
        line = line.strip() # rimuoviamo eventuali spazi e ritorni a capo

        if '<doc id="' in line:    # troviamo un nuovo documento
            doc_id, genre, gender = get_doc_info(line)
            out_file_name = f'{split}#{doc_id}#{genre}#{gender}.txt'
            out_file_path = os.path.join(out_dir, out_file_name)
            user_post = []

        elif '</doc>' in line:     # fine dell'utente
            with open(out_file_path, 'w') as out_file:
                for post in user_post:
                    out_file.write(post)
                    out_file.write('\n')

        elif line != '':  # siamo in corrispondenza del testo di un post
            user_post.append(line)

In [8]:
import os
import codecs

def convert_to_utf8(folder_path):
    # List all files in the folder
    files = os.listdir(folder_path)

    # Filter out only the text files
    text_files = [f for f in files if f.endswith('.txt')]

    # Iterate over each text file and encode it as UTF-8
    for file_name in text_files:
        file_path = os.path.join(folder_path, file_name)
        with codecs.open(file_path, 'r', encoding='utf-8', errors='ignore') as file:
            content = file.read()
        with codecs.open(file_path, 'w', encoding='utf-8') as file:
            file.write(content)

# Example usage:
folder_path = out_dir
convert_to_utf8(folder_path)

In [9]:
!pip install chardet


In [21]:
import os
import chardet

def detect_encoding(folder_path):
    # List all files in the folder
    files = os.listdir(folder_path)

    # Filter out only the text files
    text_files = [f for f in files if f.endswith('.txt')]

    # Iterate over each text file and detect its encoding
    for file_name in text_files:
        file_path = os.path.join(folder_path, file_name)
        with open(file_path, 'rb') as file:
            raw_data = file.read()
            result = chardet.detect(raw_data)
            print(f"{file_name}: {result['encoding']} (confidence: {result['confidence']})")

# Example usage:
folder_path = out_dir
detect_encoding(folder_path)


test#1#youtube#F.txt: ascii (confidence: 1.0)
test#10#youtube#F.txt: ascii (confidence: 1.0)
test#100#youtube#F.txt: ascii (confidence: 1.0)
test#1002#youtube#M.txt: ascii (confidence: 1.0)
test#1007#youtube#F.txt: ISO-8859-1 (confidence: 0.73)
test#101#youtube#F.txt: ascii (confidence: 1.0)
test#1010#youtube#F.txt: ascii (confidence: 1.0)
test#1011#youtube#M.txt: ascii (confidence: 1.0)
test#1018#youtube#F.txt: utf-8 (confidence: 0.7525)
test#102#youtube#F.txt: utf-8 (confidence: 0.87625)
test#1020#youtube#M.txt: ascii (confidence: 1.0)
test#1021#youtube#M.txt: ascii (confidence: 1.0)
test#1028#youtube#M.txt: ISO-8859-1 (confidence: 0.73)
test#103#youtube#F.txt: ascii (confidence: 1.0)
test#1030#youtube#M.txt: ISO-8859-1 (confidence: 0.5334615384615384)
test#1033#youtube#F.txt: ISO-8859-9 (confidence: 0.629823157108814)
test#1038#youtube#M.txt: ascii (confidence: 1.0)
test#104#youtube#F.txt: ascii (confidence: 1.0)
test#1048#youtube#F.txt: ascii (confidence: 1.0)
test#105#youtube#F.tx

In [20]:
parent = os.path.abspath(os.path.join(out_dir, os.pardir))
parent

'c:\\Users\\corra\\Documents\\GitHub\\ProgettoLinCompII\\Project-of-Computational-Linguistics-II\\youtube\\data'